<a href="https://colab.research.google.com/github/acoadmarmon/big-transfer-swift/blob/main/bit_swift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("tensorflow-0.11"))' Datasets ImageClassificationModels
print("\u{001B}[2J")

In [2]:
import Datasets
import ImageClassificationModels
import TensorFlow
import Python
import Foundation
import Differentiable


In [3]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

let plt = Python.import("matplotlib.pyplot")
let np  = Python.import("numpy")
let subprocess = Python.import("subprocess")
let glob = Python.import("glob")
let pil = Python.import("PIL")
let tf = Python.import("tensorflow")
//let tfp = Python.import("tensorflow_probability")
let h5py = Python.import("h5py")
let path = Python.import("os.path")

2020-11-18 03:05:34.079802: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [4]:
//subprocess.call("curl https://sdk.cloud.google.com | bash; exec -l $SHELL; gsutil ls gs://uspto-pair/applications/0800401*", shell: true)

In [5]:
let epochCount = 12
let batchSize = 32

In [6]:
// Set hyperrule parameters from bit_hyperrule.py
enum ValueError: Error {
    case invalidInput(String)
}

func get_resolution(original_resolution: (Int, Int)) -> (Int, Int) {
  let area = original_resolution.0 * original_resolution.1
  return area < 96*96 ? (160, 128) : (512, 480)
}


let known_dataset_sizes:[String: (Int, Int)] = [
  "cifar10": (32, 32),
  "cifar100": (32, 32),
  "oxford_iiit_pet": (224, 224),
  "oxford_flowers102": (224, 224),
  "imagenet2012": (224, 224),
]

func get_resolution_from_dataset(dataset: String) throws -> (Int, Int) {
  if let resolution = known_dataset_sizes[dataset] {
    return get_resolution(original_resolution: resolution)
  }
  print("Unsupported dataset " + dataset + ". Add your own here :)")
  throw ValueError.invalidInput(dataset)

}

func get_mixup(dataset_size: Int) -> Double {
  return dataset_size < 20_000 ? 0.0 : 0.1
}


func get_schedule(dataset_size: Int) -> Array<Int> {
  if dataset_size < 20_000{
    return [100, 200, 300, 400, 500]
  }
  else if dataset_size < 500_000 {
    return [500, 3000, 6000, 9000, 10_000]
  }
  else {
    return [500, 6000, 12_000, 18_000, 20_000]
  }
}


func get_lr(step: Int, dataset_size: Int, base_lr: Float = 0.003) -> Float? {
  /* Returns learning-rate for `step` or nil at the end. */
  let supports = get_schedule(dataset_size: dataset_size)
  // Linear warmup
  if step < supports[0] {
    return base_lr * Float(step) / Float(supports[0])
  }
  // End of training
  else if step >= supports.last! {
    return nil
  }
  // Staircase decays by factor of 10
  else {
    var base_lr = base_lr
    for s in supports[1...] {
      if s < step {
        base_lr = base_lr / 10.0
      }
    }
    return base_lr
  }
}

In [7]:
get_lr(step: 7000, dataset_size: 50000, base_lr: 0.01)

▿ Optional<Float>
  - some : 9.999999e-05


In [8]:
// Build model with weights
import FileManager
import PythonKit
let tf = Python.import("tensorflow")
_ = Python.builtins

var known_models = [String: String]()
// var resnetv2 = ResNetV2(classCount: 1000, depth: .resNet50)

let model_name = "BiT-M-R50x1"

struct Weights {
    let name: String
    let layer: Tensor<Float>
}

func get_pretrained_weights_dict(model_name: String) -> Array<Weights> {
  let valid_types = ["BiT-S", "BiT-M"]
  let valid_sizes = [(50, 1), (50, 3), (101, 1), (101, 3), (152, 4)]
  let bit_url = "gs://bit_models/"

  for types in valid_types {
    for sizes in valid_sizes {
      let model_string = types + "-R" + String(sizes.0) + "x" + String(sizes.1)
      known_models[model_string] = bit_url + model_string + ".npz"
    }
  }

  var f = Python.None
  
  if let model_path = known_models[model_name] {
    subprocess.call("gsutil cp " + model_path + " .", shell: true)
  }

  let weights = np.load("./" + model_name + ".npz")

  var weights_array = Array<Weights>()
  for param in weights {
      weights_array.append(Weights(name: String(param)!, layer: Tensor<Float>(numpy: weights[param])!))
  }
  return weights_array
}
var weights_array = get_pretrained_weights_dict(model_name: model_name)
// Get convolution weights

Copying gs://bit_models/BiT-M-R50x1.npz...
/ [1 files][260.4 MiB/260.4 MiB]                                                
Operation completed over 1 objects/260.4 MiB.                                    


In [9]:
import TensorFlow
func getPaddingDimensionsFromKernelSize(kernelSize: Int) -> (Int, Int) {
  let padTotal = kernelSize - 1
  let padBeginning = Int(padTotal / 2)
  let padEnd = padTotal - padBeginning
  return (kernelSize + padBeginning, kernelSize + padEnd)
}

func paddingFromKernelSize(kernelSize: Int) -> [(before: Int, after: Int)] {
  let padTotal = kernelSize - 1
  let padBeginning = Int(padTotal / 2)
  let padEnd = padTotal - padBeginning
  let padding = [
        (before: 0, after: 0),
        (before: padBeginning, after: padEnd),
        (before: padBeginning, after: padEnd),
        (before: 0, after: 0)]
  return padding
}

public struct StandardizedConv2D: Layer {
  public var conv: Conv2D<Float>

  public init(
    filterShape: (Int, Int, Int, Int),
    strides: (Int, Int) = (1, 1),
    padding: Padding = .valid,
    useBias: Bool = true
  )
  {
  self.conv = Conv2D(
      filterShape: filterShape, 
      strides: strides, 
      padding: padding,
      useBias: useBias)
  }

  @differentiable
  public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
      let axes: Array<Int> = [0, 1, 2]
      var standardizedConv = conv
      standardizedConv.filter = (standardizedConv.filter - standardizedConv.filter.mean(squeezingAxes: axes)) / sqrt((standardizedConv.filter.variance(squeezingAxes: axes) + 1e-16))
      return standardizedConv(input)
  }

}


public struct ConvGNV2: Layer {
    public var conv: StandardizedConv2D
    public var norm: GroupNorm<Float>
    @noDerivative public var isSecond: Bool

    public init(
        inFilters: Int,
        outFilters: Int,
        kernelSize: Int = 1,
        stride: Int = 1,
        padding: Padding = .valid,
        isSecond: Bool = false
    ) {
        self.conv = StandardizedConv2D(
            filterShape: (kernelSize, kernelSize, inFilters, outFilters), 
            strides: (stride, stride), 
            padding: padding,
            useBias: false)
        self.norm = GroupNorm<Float>(
              offset: Tensor(zeros: [inFilters]),
              scale: Tensor(zeros: [inFilters]),
              groupCount: 2,
              axis: -1,
              epsilon: 0.001)
        self.isSecond = isSecond
    }

    @differentiable
    public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        var normResult = norm(input)
        if self.isSecond {
            normResult = normResult.padded(forSizes: paddingFromKernelSize(kernelSize: 3))
        }
        let reluResult = relu(normResult)
        let convResult = conv(reluResult)
        return convResult
    }
}
public struct ShortcutBiT: Layer {
    public var projection: StandardizedConv2D
    public var norm: GroupNorm<Float>
    @noDerivative public let needsProjection: Bool
    
    public init(inFilters: Int, outFilters: Int, stride: Int) {
      needsProjection = (stride > 1 || inFilters != outFilters)
      norm = GroupNorm<Float>(
          offset: Tensor(zeros: [needsProjection ? inFilters  : 1]),
          scale: Tensor(zeros: [needsProjection ? inFilters  : 1]),
          groupCount: needsProjection ? 2  : 1,
          axis: -1,
          epsilon: 0.001)
        
        projection =  StandardizedConv2D(
            filterShape: (1, 1, needsProjection ? inFilters  : 1, needsProjection ? outFilters : 1), 
            strides: (stride, stride), 
            padding: .valid,
            useBias: false)
    }
    
    @differentiable
    public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        var res = input
        if needsProjection { 
          res = norm(res)
          res = relu(res)
          res = projection(res)
        }
        return res
    }
}



public struct ResidualBlockBiT: Layer {
    public var shortcut: ShortcutBiT
    public var convs: [ConvGNV2]

    public init(inFilters: Int, outFilters: Int, stride: Int, expansion: Int){
        if expansion == 1 {
            convs = [
                ConvGNV2(inFilters: inFilters,  outFilters: outFilters, kernelSize: 3, stride: stride),
                ConvGNV2(inFilters: outFilters, outFilters: outFilters, kernelSize: 3, isSecond: true)
            ]
        } else {
            convs = [
                ConvGNV2(inFilters: inFilters,    outFilters: outFilters/4),
                ConvGNV2(inFilters: outFilters/4, outFilters: outFilters/4, kernelSize: 3, stride: stride, isSecond: true),
                ConvGNV2(inFilters: outFilters/4, outFilters: outFilters)
            ]
        }
        shortcut = ShortcutBiT(inFilters: inFilters, outFilters: outFilters, stride: stride)
    }

    @differentiable
    public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        let convResult = convs.differentiableReduce(input) { $1($0) }
        return convResult + shortcut(input)
    }
}

public struct BigTransfer: Layer {
  public var inputStem: StandardizedConv2D
  public var maxPool: MaxPool2D<Float>
  public var residualBlocks: [ResidualBlockBiT] = []
  public var groupNorm : GroupNorm<Float>
  public var flatten = Flatten<Float>()
  public var classifier: Dense<Float>
  public var avgPool = GlobalAvgPool2D<Float>()
  @noDerivative public var finalOutFilter : Int = 0

  public init(
        classCount: Int, 
        depth: Depth, 
        inputChannels: Int = 3
    ) {

        self.inputStem = StandardizedConv2D(filterShape: (7, 7, 3, 64), strides: (2, 2), padding: .valid, useBias: false)
        self.maxPool = MaxPool2D(poolSize: (3, 3), strides: (2, 2), padding: .valid)
        let sizes = [64 / depth.expansion, 64, 128, 256, 512]
        for (iBlock, nBlocks) in depth.layerBlockSizes.enumerated() {
            let (nIn, nOut) = (sizes[iBlock] * depth.expansion, sizes[iBlock+1] * depth.expansion)
            for j in 0..<nBlocks {

                self.residualBlocks.append(ResidualBlockBiT(
                    inFilters: j==0 ? nIn : nOut,  
                    outFilters: nOut, 
                    stride: (iBlock != 0) && (j == 0) ? 2 : 1, 
                    expansion: depth.expansion
                ))
                self.finalOutFilter = nOut
            }
        }
        self.groupNorm = GroupNorm<Float>(
              offset: Tensor(zeros: [self.finalOutFilter]),
              scale: Tensor(zeros: [self.finalOutFilter]),
              groupCount: 2,
              axis: -1,
              epsilon: 0.001)
        self.classifier = Dense(inputSize: 512 * depth.expansion, outputSize: classCount)
    }

  @differentiable
  public func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
      var paddedInput = input.padded(forSizes: paddingFromKernelSize(kernelSize: 7))
      paddedInput = inputStem(paddedInput).padded(forSizes: paddingFromKernelSize(kernelSize: 3))
      let inputLayer = maxPool(paddedInput)
      let blocksReduced = residualBlocks.differentiableReduce(inputLayer) { $1($0) }
      let normalized = relu(groupNorm(blocksReduced))
      return normalized.sequenced(through: avgPool, flatten, classifier)
  }
}

extension BigTransfer {
    public enum Depth {
        case resNet18
        case resNet34
        case resNet50
        case resNet101
        case resNet152

        var expansion: Int {
            switch self {
            case .resNet18, .resNet34: return 1
            default: return 4
            }
        }

        var layerBlockSizes: [Int] {
            switch self {
            case .resNet18:  return [2, 2, 2,  2]
            case .resNet34:  return [3, 4, 6,  3]
            case .resNet50:  return [3, 4, 6,  3]
            case .resNet101: return [3, 4, 23, 3]
            case .resNet152: return [3, 8, 36, 3]
            }
        }
    }
}

In [10]:
func get_model_units(model_name: String) -> BigTransfer.Depth {
  if model_name.contains("R50") {
    return .resNet50
  }
  else if model_name.contains("R101") {
    return .resNet101
  }
  else {
    return .resNet152
  }
}

let bit_type = get_model_units(model_name: model_name)

In [11]:
var resnetv2 = BigTransfer(classCount: 10, depth: .resNet50)

let convs = weights_array.filter {key in return key.name.contains("/block") && key.name.contains("standardized_conv2d/kernel") && !(key.name.contains("proj"))}

var k = 0
for (idx, i) in resnetv2.residualBlocks.enumerated() {
  for (jdx, _) in i.convs.enumerated() {
    assert(resnetv2.residualBlocks[idx].convs[jdx].conv.conv.filter.shape == convs[k].layer.shape)
    resnetv2.residualBlocks[idx].convs[jdx].conv.conv.filter = convs[k].layer
    k = k + 1
  }
}
print(convs.count, k)
// https://github.com/zaidalyafeai/Swift4TF/blob/master/Swift4TF_TransferLearning.ipynb 

48 48


In [12]:
// Set weights for all projective convolutions
let projective_convs = weights_array.filter {key in return key.name.contains("/block") && key.name.contains("standardized_conv2d/kernel") && (key.name.contains("proj"))}
let norm_scale = weights_array.filter {key in return key.name.contains("unit01/a/group_norm/gamma")}
let norm_offset = weights_array.filter {key in return key.name.contains("unit01/a/group_norm/beta")}

var k = 0
for (idx, i) in resnetv2.residualBlocks.enumerated() {
    if (i.shortcut.projection.conv.filter.shape != [1, 1, 1, 1])
    {
      assert(resnetv2.residualBlocks[idx].shortcut.projection.conv.filter.shape == projective_convs[k].layer.shape)
      resnetv2.residualBlocks[idx].shortcut.projection.conv.filter = projective_convs[k].layer

      assert(resnetv2.residualBlocks[idx].shortcut.norm.scale.shape == norm_scale[k].layer.shape)
      resnetv2.residualBlocks[idx].shortcut.norm.scale = norm_scale[k].layer

      assert(resnetv2.residualBlocks[idx].shortcut.norm.offset.shape == norm_offset[k].layer.shape)
      resnetv2.residualBlocks[idx].shortcut.norm.offset = norm_offset[k].layer
      k = k + 1
    }
}
print(projective_convs.count, k)

4 4


In [13]:
// Set weights for all standard norms
let norm_scale = weights_array.filter {key in return key.name.contains("gamma")}

var k = 0
for (idx, i) in resnetv2.residualBlocks.enumerated() {
  for (jdx, _) in i.convs.enumerated() {
    assert(norm_scale[k].layer.shape == resnetv2.residualBlocks[idx].convs[jdx].norm.scale.shape)
    resnetv2.residualBlocks[idx].convs[jdx].norm.scale = norm_scale[k].layer
    k = k + 1
  }
}

let norm_offset = weights_array.filter {key in return key.name.contains("beta")}

var l = 0
for (idx, i) in resnetv2.residualBlocks.enumerated() {
  for (jdx, _) in i.convs.enumerated() {
    assert(norm_offset[l].layer.shape == resnetv2.residualBlocks[idx].convs[jdx].norm.offset.shape)
    resnetv2.residualBlocks[idx].convs[jdx].norm.offset = norm_offset[l].layer
    l = l + 1
  }
}

assert(resnetv2.groupNorm.scale.shape == norm_scale[k].layer.shape)
resnetv2.groupNorm.scale = norm_scale[k].layer
assert(resnetv2.groupNorm.offset.shape == norm_offset[l].layer.shape)
resnetv2.groupNorm.offset = norm_offset[l].layer

In [15]:
// Set weight for input block
let root_convs = weights_array.filter {key in return key.name.contains("root_block")}
assert(resnetv2.inputStem.conv.filter.shape == root_convs[0].layer.shape)
resnetv2.inputStem.conv.filter = root_convs[0].layer

In [16]:
let norm_scale = weights_array.filter {key in return key.name.contains("unit01/a/group_norm/gamma")}
print(norm_scale.count, projective_convs.count)
for i in projective_convs {
  print(i.name)
}



4 4
resnet/block1/unit01/a/proj/standardized_conv2d/kernel
resnet/block2/unit01/a/proj/standardized_conv2d/kernel
resnet/block3/unit01/a/proj/standardized_conv2d/kernel
resnet/block4/unit01/a/proj/standardized_conv2d/kernel


In [17]:
let norm_scale = weights_array.filter {key in return key.name.contains("dense")}
norm_scale

0 elements


In [18]:
let _ = _ExecutionContext.global

let device = Device.defaultXLA
device

▿ Device(kind: .GPU, ordinal: 0, backend: .XLA)
  - kind : TensorFlow.Device.Kind.GPU
  - ordinal : 0
  - backend : TensorFlow.Device.Backend.XLA


In [19]:
resnetv2.move(to: device)
var optimizer = SGD(for: resnetv2, learningRate: 0.001, momentum: 0.9)
optimizer = SGD(copying: optimizer, to: device)

let cifar10_training_size = 50000

Context.local.learningPhase = .training
let batchSize = 128
let lrSupports = get_schedule(dataset_size: cifar10_training_size)
let scheduleLength = lrSupports.last!
let stepsPerEpoch = cifar10_training_size / batchSize


var epochCount = scheduleLength / stepsPerEpoch
let dataset = CIFAR10(batchSize: batchSize, on: Device.default)

print(stepsPerEpoch)
print(epochCount)

390
25


In [20]:
struct Statistics {
    var correctGuessCount = Tensor<Int32>(0, on: Device.default)
    var totalGuessCount = Tensor<Int32>(0, on: Device.default)
    var totalLoss = Tensor<Float>(0, on: Device.default)
    var batches: Int = 0
    var accuracy: Float { 
        Float(correctGuessCount.scalarized()) / Float(totalGuessCount.scalarized()) * 100 
    } 
    var averageLoss: Float { totalLoss.scalarized() / Float(batches) }

    init(on device: Device = Device.default) {
        correctGuessCount = Tensor<Int32>(0, on: device)
        totalGuessCount = Tensor<Int32>(0, on: device)
        totalLoss = Tensor<Float>(0, on: device)
    }

    mutating func update(logits: Tensor<Float>, labels: Tensor<Float>, loss: Tensor<Float>) {
        let correct = logits.argmax(squeezingAxis: 1) .== labels.argmax(squeezingAxis: 1)
        correctGuessCount += Tensor<Int32>(correct).sum()
        totalGuessCount += Int32(labels.shape[0])
        totalLoss += loss
        batches += 1
    }
}




In [ ]:
print("Beginning training...")
var curr_step: Int = 1
let resize_size = get_resolution(original_resolution: (32, 32))
var mixup_alpha = get_mixup(dataset_size: cifar10_training_size)
let beta = np.random.beta(mixup_alpha, mixup_alpha)

for (epoch, batches) in dataset.training.prefix(epochCount).enumerated() {
    let start = Date()
    var trainStats = Statistics(on: device)
    var testStats = Statistics(on: device)
    
    Context.local.learningPhase = .training
    for batch in batches {
        if let new_lr = get_lr(step: curr_step, dataset_size: cifar10_training_size, base_lr: 0.003) {
          optimizer.learningRate = new_lr
          curr_step = curr_step + 1
        }
        else {
          continue
        }

        var (eagerImages, eagerLabels) = (batch.data, batch.label)
        var npImages = eagerImages.makeNumpyArray()
        var resized = tf.image.resize(npImages, [resize_size.0, resize_size.0])
        var cropped = tf.image.random_crop(resized, [batchSize, resize_size.1, resize_size.1, 3])
        var flipped = tf.image.random_flip_left_right(cropped)
        var mixed_up = flipped
        var newLabels = Tensor<Float>(Tensor<Int32>(oneHotAtIndices: eagerLabels, depth: 10))
        if mixup_alpha > 0.0 {
          var npLabels = newLabels.makeNumpyArray()
          mixed_up = beta * mixed_up + (1 - beta) * tf.reverse(mixed_up, axis: [0])
          npLabels = beta * npLabels + (1 - beta) * tf.reverse(npLabels, axis: [0])
          newLabels = Tensor<Float>(numpy: npLabels.numpy())!
        }
        eagerImages = Tensor<Float>(numpy: mixed_up.numpy())!
        let images = Tensor(copying: eagerImages, to: device)
        let labels = Tensor(copying: newLabels, to: device)

        let 𝛁model = TensorFlow.gradient(at: resnetv2) { resnetv2 -> Tensor<Float> in
            let ŷ = resnetv2(images)
            let loss = softmaxCrossEntropy(logits: ŷ, probabilities: labels)
            trainStats.update(logits: ŷ, labels: labels, loss: loss)
            return loss
        }

        
        optimizer.update(&resnetv2, along: 𝛁model)
        LazyTensorBarrier()
    }

    Context.local.learningPhase = .inference
    for batch in dataset.validation {
        var (eagerImages, eagerLabels) = (batch.data, batch.label)
        var npImages = eagerImages.makeNumpyArray()
        var resized = tf.image.resize(npImages, [resize_size.0, resize_size.0])
        eagerImages = Tensor<Float>(numpy: resized.numpy())!
        var newLabels = Tensor<Float>(Tensor<Int32>(oneHotAtIndices: eagerLabels, depth: 10))
        let images = Tensor(copying: eagerImages, to: device)
        let labels = Tensor(copying: newLabels, to: device)
        let ŷ = resnetv2(images)
        let loss = softmaxCrossEntropy(logits: ŷ, probabilities: labels)
        LazyTensorBarrier()
        testStats.update(logits: ŷ, labels: labels, loss: loss)
    }

    print(
        """
        [Epoch \(epoch)] \
        Training Loss: \(String(format: "%.3f", trainStats.averageLoss)), \
        Training Accuracy: \(trainStats.correctGuessCount)/\(trainStats.totalGuessCount) \
        (\(String(format: "%.1f", trainStats.accuracy))%), \
        Test Loss: \(String(format: "%.3f", testStats.averageLoss)), \
        Test Accuracy: \(testStats.correctGuessCount)/\(testStats.totalGuessCount) \
        (\(String(format: "%.1f", testStats.accuracy))%) \
        seconds per epoch: \(String(format: "%.1f", Date().timeIntervalSince(start)))
        """)
}


Beginning training...
2020-11-18 03:06:16.838107: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-18 03:06:16.838328: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-18 03:06:16.838990: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-11-18 03:06:16.839061: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-18 03:06:16.839113: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2020-11-18 03:06:16.839149: I tensorflow/stream_execu

In [23]:
// Try doing SWA on stable LR

"""
On ImageNet we experimented with ResNet-50, ResNet-152[He et al., 2016] and DenseNet-161[Huang et al.,2017].  
For these architectures we used pretrained mod-els  fromPyTorch.torchvision.   For  each  of  themodels we ran 
SWA for10epochs with a cyclical learn-ing rate schedule with the same parameters for all models
(the details can be found in the Appendix), 
and report themean and standard deviation of test error averaged over3runs. The results are shown in Table 2.
"""
// Run 10 epochs of cyclical LR at the end here with SWA
// https://arxiv.org/pdf/1803.05407.pdf
// https://pytorch.org/blog/stochastic-weight-averaging-in-pytorch/
// https://github.com/izmailovpavel/contrib_swa_examples/blob/master/train.py

"On ImageNet we experimented with ResNet-50, ResNet-152[He et al., 2016] and DenseNet-161[Huang et al.,2017].  \nFor these architectures we used pretrained mod-els  fromPyTorch.torchvision.   For  each  of  themodels we ran \nSWA for10epochs with a cyclical learn-ing rate schedule with the same parameters for all models\n(the details can be found in the Appendix), \nand report themean and standard deviation of test error averaged over3runs. The results are shown in Table 2."


In [24]:
func get_cosine_annealing_lr(step: Int, steps_per_epoch: Int, base_lr: Float = 0.003, max_lr: Float = 0.1) -> Float? {
    return base_lr + 0.5*(max_lr - base_lr)*(1 + cos(Float.pi * Float(step)/Float(steps_per_epoch)))
}
get_cosine_annealing_lr(step: 1, steps_per_epoch: 100)

▿ Optional<Float>
  - some : 0.09997606


In [25]:
var swaModelWeights: BigTransfer.TangentVector = BigTransfer.TangentVector.zero
swaModelWeights = BigTransfer.TangentVector(copying: swaModelWeights, to: device)

var numCycles = 10
for (epoch, batches) in dataset.training.prefix(numCycles).enumerated() {
    let start = Date()
    var trainStats = Statistics(on: device)
    var testStats = Statistics(on: device)
    
    var currStep: Int = 0
    Context.local.learningPhase = .training
    for batch in batches {
        if let new_lr = get_cosine_annealing_lr(step: currStep, steps_per_epoch: stepsPerEpoch, base_lr: 0.0001, max_lr: 0.001) {
          optimizer.learningRate = new_lr
          curr_step = curr_step + 1
        }
        else {
          continue
        }

        var (eagerImages, eagerLabels) = (batch.data, batch.label)
        var npImages = eagerImages.makeNumpyArray()
        var resized = tf.image.resize(npImages, [resize_size.0, resize_size.0])
        var cropped = tf.image.random_crop(resized, [batchSize, resize_size.1, resize_size.1, 3])
        var flipped = tf.image.random_flip_left_right(cropped)
        var mixed_up = flipped
        var newLabels = Tensor<Float>(Tensor<Int32>(oneHotAtIndices: eagerLabels, depth: 10))
        if mixup_alpha > 0.0 {
          var npLabels = newLabels.makeNumpyArray()
          mixed_up = beta * mixed_up + (1 - beta) * tf.reverse(mixed_up, axis: [0])
          npLabels = beta * npLabels + (1 - beta) * tf.reverse(npLabels, axis: [0])
          newLabels = Tensor<Float>(numpy: npLabels.numpy())!
        }
        eagerImages = Tensor<Float>(numpy: mixed_up.numpy())!
        let images = Tensor(copying: eagerImages, to: device)
        let labels = Tensor(copying: newLabels, to: device)
        let 𝛁model = TensorFlow.gradient(at: resnetv2) { resnetv2 -> Tensor<Float> in
            let ŷ = resnetv2(images)
            let loss = softmaxCrossEntropy(logits: ŷ, probabilities: labels)
            trainStats.update(logits: ŷ, labels: labels, loss: loss)
            return loss
        }

        optimizer.update(&resnetv2, along: 𝛁model)
        LazyTensorBarrier()

        currStep = currStep + 1
        if currStep == stepsPerEpoch - 1 {
          print("Adding model weights")
          swaModelWeights = swaModelWeights + 𝛁model
        }
    }

    Context.local.learningPhase = .inference
    for batch in dataset.validation {
        var (eagerImages, eagerLabels) = (batch.data, batch.label)
        var npImages = eagerImages.makeNumpyArray()
        var resized = tf.image.resize(npImages, [resize_size.0, resize_size.0])
        eagerImages = Tensor<Float>(numpy: resized.numpy())!
        var newLabels = Tensor<Float>(Tensor<Int32>(oneHotAtIndices: eagerLabels, depth: 10))
        let images = Tensor(copying: eagerImages, to: device)
        let labels = Tensor(copying: newLabels, to: device)
        let ŷ = resnetv2(images)
        let loss = softmaxCrossEntropy(logits: ŷ, probabilities: labels)
        LazyTensorBarrier()
        testStats.update(logits: ŷ, labels: labels, loss: loss)
    }

    print(
        """
        [Epoch \(epoch)] \
        Training Loss: \(String(format: "%.3f", trainStats.averageLoss)), \
        Training Accuracy: \(trainStats.correctGuessCount)/\(trainStats.totalGuessCount) \
        (\(String(format: "%.1f", trainStats.accuracy))%), \
        Test Loss: \(String(format: "%.3f", testStats.averageLoss)), \
        Test Accuracy: \(testStats.correctGuessCount)/\(testStats.totalGuessCount) \
        (\(String(format: "%.1f", testStats.accuracy))%) \
        seconds per epoch: \(String(format: "%.1f", Date().timeIntervalSince(start)))
        """)
}

Adding model weights
[Epoch 0] Training Loss: 3.373, Training Accuracy: 9053/49920 (18.1%), Test Loss: 2.163, Test Accuracy: 1983/10000 (19.8%) seconds per epoch: 303.4
Adding model weights
[Epoch 1] Training Loss: 2.106, Training Accuracy: 11060/49920 (22.2%), Test Loss: 2.077, Test Accuracy: 2305/10000 (23.0%) seconds per epoch: 307.1
Adding model weights
[Epoch 2] Training Loss: 2.067, Training Accuracy: 11664/49920 (23.4%), Test Loss: 2.035, Test Accuracy: 2461/10000 (24.6%) seconds per epoch: 278.8
Adding model weights
[Epoch 3] Training Loss: 2.031, Training Accuracy: 12311/49920 (24.7%), Test Loss: 2.001, Test Accuracy: 2561/10000 (25.6%) seconds per epoch: 283.6
Adding model weights
[Epoch 4] Training Loss: 2.011, Training Accuracy: 12890/49920 (25.8%), Test Loss: 1.991, Test Accuracy: 2651/10000 (26.5%) seconds per epoch: 285.5


In [26]:
var averagedModelWeights = swaModelWeights
averagedModelWeights.inputStem.conv.filter = swaModelWeights.inputStem.conv.filter/Float(numCycles)
averagedModelWeights.groupNorm.offset = swaModelWeights.groupNorm.offset/Float(numCycles)
averagedModelWeights.groupNorm.scale = swaModelWeights.groupNorm.scale/Float(numCycles)
averagedModelWeights.classifier.weight = swaModelWeights.classifier.weight/Float(numCycles)
averagedModelWeights.classifier.bias = swaModelWeights.classifier.bias/Float(numCycles)

for (index, weight)  in swaModelWeights.residualBlocks.enumerated() {
  averagedModelWeights.residualBlocks[index].shortcut.projection.conv.filter = weight.shortcut.projection.conv.filter/Float(numCycles)
  averagedModelWeights.residualBlocks[index].shortcut.norm.offset = weight.shortcut.norm.offset/Float(numCycles)
  averagedModelWeights.residualBlocks[index].shortcut.norm.scale = weight.shortcut.norm.scale/Float(numCycles)
     for (conv_index, conv_weight) in weight.convs.enumerated() {
       averagedModelWeights.residualBlocks[index].convs[conv_index].conv.conv.filter = conv_weight.conv.conv.filter/Float(numCycles)
       averagedModelWeights.residualBlocks[index].convs[conv_index].norm.offset = conv_weight.norm.offset/Float(numCycles)
       averagedModelWeights.residualBlocks[index].convs[conv_index].norm.scale = conv_weight.norm.scale/Float(numCycles)
     }

}

In [27]:
optimizer.update(&resnetv2, along: averagedModelWeights)

In [28]:
var testStats = Statistics(on: device)
Context.local.learningPhase = .inference
for batch in dataset.validation {
    var (eagerImages, eagerLabels) = (batch.data, batch.label)
    var npImages = eagerImages.makeNumpyArray()
    var resized = tf.image.resize(npImages, [resize_size.0, resize_size.0])
    eagerImages = Tensor<Float>(numpy: resized.numpy())!
    var newLabels = Tensor<Float>(Tensor<Int32>(oneHotAtIndices: eagerLabels, depth: 10))
    let images = Tensor(copying: eagerImages, to: device)
    let labels = Tensor(copying: newLabels, to: device)
    let ŷ = resnetv2(images)
    let loss = softmaxCrossEntropy(logits: ŷ, probabilities: labels)
    LazyTensorBarrier()
    testStats.update(logits: ŷ, labels: labels, loss: loss)
}
print(
  """
  Test Accuracy: \(testStats.correctGuessCount)/\(testStats.totalGuessCount)
  """)

Test Accuracy: 2642/10000
